<a href="https://colab.research.google.com/github/Grashch/Kaggle_tasks/blob/main/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [41]:
data = pd.read_csv('/content/melb_data.csv')
y = data['Price']
X = data.drop('Price', axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=24)

In [42]:
c_cols = [col for col in X_train.columns if X_train[col].dtype == 'object' and X_train[col].nunique() < 20]
n_cols = [col for col in X_train.columns if X_train[col].dtype in ['float64', 'int64']]
cols = c_cols + n_cols
X_train = X_train[cols]
X_valid = X_valid[cols]

In [43]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [44]:
num_transformer = SimpleImputer(strategy='median')

cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_transformer, n_cols),
    ('cat', cat_transformer, c_cols)
])

In [45]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [46]:
model = RandomForestRegressor(n_estimators=100, random_state=1)

In [47]:
my_preprocessor = Pipeline([
    ('preprocessing', preprocessor),
    ('modeling', model)
])

In [48]:
my_preprocessor.fit(X_train, y_train)
y_pred = my_preprocessor.predict(X_valid)
score = mean_absolute_error(y_valid, y_pred)
print(f'MAE: {score}')

MAE: 165306.2398754003
